In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
import shutil

In [ ]:
data_dir = os.getcwd()
img_dir = os.path.join(data_dir, 'images/')
excel_path = os.path.join(data_dir, 'hh_video_data_v2.xlsx')
label_path = os.path.join(data_dir, 'handhygiene_label.csv') 

df_ex = pd.read_excel(excel_path)

In [ ]:
"""
Seperating raw images into 3 categories; surgery, anesthesia, still(useless images) 
"""
# classes directories
# /images
### /surgery
### /anesthesia
###    /handhygiene (target)
### /still

def sort_frames_into_categories(img_dir):
    cnt = 0
    for image in tqdm(glob(img_dir+'*.jpg')):
        img_name= image.split('/')[-1].split('.')[0]
        
        video_id = int(img_name.split('_')[0])
        frame_num = img_name[-6:] # ex: 000001
        
        # classes : surgery, still, target(handhygiene)
        df = df_ex[df_ex['video_id'] == video_id]
        surgery_list = df['surgery_frame'].iloc[0]
        still_list = df['still_frame'].iloc[0]
        target_list = df['target_frame'].iloc[0]
        
        frames = {'surgery':surgery_list, 'still':still_list, 'anesthesia/handhygiene':target_list}
        
        
        for procedure in ['surgery', 'still', 'anesthesia/handhygiene']:
            if frames[procedure] is np.nan:
                continue
                
            lists = frames[procedure].strip().split(',')
            for el in lists:
                start = el.strip().split('-')[0]
                end = el.strip().split('-')[1]

                # if current frame is in range of (start, end)
                if int(frame_num) in range(int(start), int(end)+1):
                    shutil.move(image, os.path.join(data_dir, 'images/'+procedure))
                    cnt += 1
                    break
    
    print("{} images moved to {}".format(cnt, os.path.join(data_dir, 'images/'+procedure)))
    #!mv /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/labeled/* /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/images/anesthesia/
    return

In [ ]:
sort_frames_into_categories(img_dir)